### Лекция 4 - Элементы функционального программирования в Python

- синтаксис функций
- LEGB
- лямбды
- filter, map, zip
- генераторы и итераторы
- декораторы
- модуль functools
- модуль itertools (is back)

In [1]:
# пример простейших функций
# (уже не ново, т.к. видели примеры в пакете algo)

def foo(x):
    return x + 10

def bar():
    print('Hello')
    
magic_number = foo(63)
print(magic_number)

bar()

73
Hello


In [25]:
# функция может возвращать все что угодно:
# удобно, т.к. можно вернуть несколько значений сразу
# (по сути кортеж)
def arithmetics(a, b):
    ''' функция возвращает сумму, разность и произведение чисел '''
    summa = a + b
    diff = a - b
    mult = a * b
    return summa, diff, mult

s = arithmetics(10, 15)
s

(25, -5, 150)

In [31]:
_, difference, _ = arithmetics(7, 9)
difference

-2

In [2]:
# Синтаксис *args, **kwargs:
def avg(a, *args, **kwargs):
    print('args:')
    print(args)
    print('kwargs:')
    print(kwargs)
    
avg(1, 2, 3, 'Hello', par1=0, par2=-1)

args:
(2, 3, 'Hello')
kwargs:
{'par2': -1, 'par1': 0}


In [19]:
param_list = [1, 2, 3]
avg('Hello!', param_list, par1=0, par2=-1)

args:
([1, 2, 3],)
kwargs:
{'par2': -1, 'par1': 0}


In [20]:
avg('Hello!', *param_list, par1=0, par2=-1)

args:
(1, 2, 3)
kwargs:
{'par2': -1, 'par1': 0}


In [3]:
# очень важно: функция - это first-class объект!
dir(avg)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [4]:
# что-нибудь поинстроспектируем:
print('Qualified name:', avg.__qualname__)
print('Vars:', avg.__code__.co_varnames)

Qualified name: avg
Vars: ('a', 'args', 'kwargs')


In [5]:
# эквивалентно foo(10)
foo.__call__(10)

20

In [6]:
# таким образом, функции можно присваивать переменным:
cooler_foo = foo
cooler_foo(100)

110

In [7]:
# можно делать массив из функций
# и последовательно выполнять их:
import math

funcs = [foo, math.factorial]

for func in funcs:
    print(func.__qualname__, func(5))

foo 15
factorial 120


In [10]:
# можно передавать функцию в функцию:
def process_elementwise(seq, func):
    new_seq = [func(s) for s in seq]
    return new_seq

words = ['Fundamentals', 'of', 'brainwashing...']
process_elementwise(words, str.__len__)

[12, 2, 15]

In [13]:
process_elementwise(words, str.upper)

['FUNDAMENTALS', 'OF', 'BRAINWASHING...']

In [16]:
nums = [1, -3, 5, -7, -9]
process_elementwise(nums, abs)

[1, 3, 5, 7, 9]

In [17]:
process_elementwise(nums, foo)

[11, 7, 15, 3, 1]

In [8]:
# Посчитаем сумму цифр числа в декларативном стиле:
n = 123
print(sum(map(int, str(n))))

6


In [33]:
from functools import reduce

s = range(1, 5)
reduce(lambda x, y: x + y, s)

10

### Итераторы и генераторы:
- итератор - это концепция (любой объект, имеющий методы ```next()``` и ```__iter__()```)
- генератор - языковое средство (объект вокруг функции с ```yield```)

Любой генератор является итератором, но не наоборот.

In [37]:
# еще пример генератора (генерирующего цифры числа)
def generate_digits(x):
    while x > 0:
        yield x % 10
        x = x // 10

for digit in generate_digits(5716):
    print(digit)

6
1
7
5


In [39]:
l = [x for x in generate_digits(54321)]
l

[1, 2, 3, 4, 5]

In [40]:
i = generate_digits(4523)
print([next(i), next(i), next(i)])

[3, 2, 5]


In [35]:
# Декораторы

def decorate(f):
    def new(*args, **kwargs):
        print(" ======= BEGIN ")
        f(args[0])
        print(" ======= END ")

    # декоратор возвращает функцию (callable)
    return new

# декорируем функцию
@decorate
def hello(x):
    print(x)

hello("OK")

# происходит такой вызов:
# decorate(hello("OK"))()

 ======= BEGIN 
OK
 ======= END 
